In [17]:
import pathlib

import pandas as pd
import numpy as np
from scipy.io import arff
from gosdt.model.threshold_guess import compute_thresholds, cut
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from imodels.tree.gosdt.pygosdt import OptimalTreeClassifier
import random
import torch
import sklearn
from gosdt import GOSDT
from metatree.model_metatree import LlamaForMetaTree as MetaTree
from metatree.decision_tree_class import DecisionTree, DecisionTreeForest
from metatree.run_train import preprocess_dimension_patch

from transformers import AutoConfig

model_name_or_path = "yzhuang/MetaTree"

config = AutoConfig.from_pretrained(model_name_or_path)
model = MetaTree.from_pretrained(
    model_name_or_path,
    config=config,
)   
decision_tree_forest = DecisionTreeForest()

ensemble_size = 1
seed = 42
# Load the new dataset
arff_file = arff.loadarff('./dataset/17.wind.arff')
df = pd.DataFrame(arff_file[0])
print(df)
df['Result'] = df['binaryClass'].apply(lambda x: 1 if x == b'P' else 0)

# Drop the original binaryClass column
df.drop(columns=['binaryClass'], inplace=True)
X, y = df.iloc[:,:-1], df.iloc[:,-1]
df.info()
df = df.astype(int)
df.info()
feature_names = df.columns[:-1]

print("Dataset Shapes X={}, y={}, Num of Classes={}".format(X.shape, y.shape, len(set(y))))

train_idx, test_idx = sklearn.model_selection.train_test_split(range(X.shape[0]), test_size=0.3, random_state=seed)
print(len(train_idx))

# Dimension 
random.seed(seed)
feature_idx = np.random.choice(X.shape[1], 6, replace=False)
X = X.iloc[:, feature_idx]  # Updated line

test_X, test_y = X.iloc[test_idx], y.iloc[test_idx]  # Updated line

for i in range(ensemble_size):
    # Sample Train and Test Data
    random.seed(seed+i+1)
    subset_idx = random.sample(train_idx, 256)
    train_X, train_y = X.iloc[subset_idx], y.iloc[subset_idx]  # Updated line

    input_x = torch.tensor(train_X.values, dtype=torch.float32)  # Convert DataFrame to NumPy array before tensor
    input_y = torch.nn.functional.one_hot(torch.tensor(train_y.values, dtype= torch.long)).float()  # Convert Series to NumPy array before tensor

    batch = {"input_x": input_x, "input_y": input_y, "input_y_clean": input_y}
    batch = preprocess_dimension_patch(batch, n_feature=10, n_class=10)
    model.depth = 2
    outputs = model.generate_decision_tree(batch['input_x'], batch['input_y'], depth=model.depth)
    decision_tree_forest.add_tree(DecisionTree(auto_dims=outputs.metatree_dimensions, auto_thresholds=outputs.tentative_splits, input_x=batch['input_x'], input_y=batch['input_y'], depth=model.depth))

    print("Decision Tree Features: ", [x.argmax(dim=-1) for x in outputs.metatree_dimensions])
    print("Decision Tree Threasholds: ", outputs.tentative_splits)


      year  month   day    RPT    VAL    ROS    KIL    SHA    BIR    DUB  \
0     61.0    1.0   1.0  15.04  14.96  13.17   9.29  13.96   9.87  13.67   
1     61.0    1.0   2.0  14.71  16.88  10.83   6.50  12.62   7.67  11.50   
2     61.0    1.0   3.0  18.50  16.88  12.33  10.13  11.17   6.17  11.25   
3     61.0    1.0   4.0  10.58   6.63  11.75   4.58   4.54   2.88   8.63   
4     61.0    1.0   5.0  13.33  13.25  11.42   6.17  10.71   8.21  11.92   
...    ...    ...   ...    ...    ...    ...    ...    ...    ...    ...   
6569  78.0   12.0  27.0  17.58  16.96  17.62   8.08  13.21  11.67  14.46   
6570  78.0   12.0  28.0  13.21   5.46  13.46   5.00   8.12   9.42  14.33   
6571  78.0   12.0  29.0  14.00  10.29  14.42   8.71   9.71  10.54  19.17   
6572  78.0   12.0  30.0  18.50  14.04  21.29   9.13  12.75   9.71  18.08   
6573  78.0   12.0  31.0  20.33  17.41  27.29   9.59  12.08  10.13  19.25   

        CLA    MUL    CLO    BEL binaryClass  
0     10.25  10.83  12.58  18.50        

In [18]:
tree_pred = decision_tree_forest.predict(torch.tensor(test_X.values, dtype=torch.float32))

accuracy = accuracy_score(test_y.values, tree_pred.argmax(dim=-1).squeeze(0))
print("MetaTree Test Accuracy: ", accuracy)


MetaTree Test Accuracy:  0.7557019766852509


In [19]:
cart_ensemble = []

for i in range(ensemble_size):
    random.seed(seed + i + 1)
    subset_idx = random.sample(train_idx, 256)
    train_X, train_y = X.iloc[subset_idx], y.iloc[subset_idx]  # Use .iloc for row indexing
    
    clf = sklearn.tree.DecisionTreeClassifier(max_depth=2, random_state=seed + i + 1)
    clf.fit(train_X.values, train_y.values)
    cart_ensemble.append(clf)

overall_pred = np.zeros((test_X.values.shape[0], len(set(test_y))))
for clf in cart_ensemble:
    overall_pred += clf.predict_proba(test_X.values)
overall_pred = overall_pred / len(cart_ensemble)

accuracy = accuracy_score(test_y.values, overall_pred.argmax(axis=-1))
print("CART Test Accuracy: ", accuracy)


CART Test Accuracy:  0.7136340598073999


In [20]:
# # Tiếp tục từ đoạn code của bạn
# # Định nghĩa các tham số cho mô hình GOSDT
# gosdt_params = {
#     'regularization': 0.01,
#     'time_limit': 60,  # giới hạn thời gian tính toán là 60 giây
#     'verbose': True
# }
# 
# # Khởi tạo mô hình GOSDT với các tham số đã định nghĩa
# gosdt_model = GOSDT(gosdt_params)
# 
# # Huấn luyện mô hình GOSDT với tập dữ liệu huấn luyện
# gosdt_model.fit(train_X, train_y)
# 
# # Dự đoán kết quả trên tập dữ liệu kiểm tra
# predictions = gosdt_model.predict(test_X)
# 
# # Tính toán độ chính xác của mô hình
# accuracy = accuracy_score(test_y, predictions)
# print(f'Độ chính xác của mô hình GOSDT: {accuracy}')


In [21]:
# gosdt_ensemble = []
# 
# for i in range(ensemble_size):
#     random.seed(seed + i + 1)
#     subset_idx = random.sample(train_idx, 256)
#     train_X, train_y = X.iloc[subset_idx], y.iloc[subset_idx]  # Use .iloc for row indexing
#     glf = OptimalTreeClassifier(random_state=seed + i + 1)
#     print(glf.max_depth())
#     gosdt_ensemble.append(predictions)
# overall_pred_gosdt = np.zeros((test_X.values.shape[0], len(set(test_y))))
# for glf in gosdt_ensemble:
#     overall_pred_gosdt += glf.predict_proba(test_X.values)
# overall_pred_gosdt = overall_pred_gosdt / len(gosdt_ensemble)
# 
# accuracy = accuracy_score(test_y.values, overall_pred_gosdt.argmax(axis=-1))
# print("GOSDT Test Accuracy: ", accuracy)
#     